In [1]:
import sys, os 

import numpy as np
import pandas as pd

import scipy as sci
import xarray as xr
from pathlib import Path

import csv
    
from tqdm import tqdm

# Growth Model for one location

In [32]:
# !! North Atlantic

""" Definitions of growth model constants from Butzin and Pörtner, 2016  """

A_R = 8.660        # Rate of uninhibited growth at reference temperature T_R (% d^-1 g^1/b)
B_R = 0.3055       # Value of allometric exponent at reference temperature Tr
THETA_A = 18145    # Arrhenius temperature (K) for uninhibited reaction kinetics = 17871,85°C
THETA_B = 4258     # Arrhenius temperature (K) = 3984,85°C 
THETA_H =  25234   # Arrhenius temperature (K) for inhibited reaction kinetics = 24960,85°C
T_R = 283          # Reference optimum temperature (K) = 9.85°C
T_H = 286          # Temperature for inhibitive processes (K) = 12.85°C
C_AVG =  0.291     # Independent of temperature and weight constant (% d^-1)

""" Definitions of kinetic functions """
Kelvin = 273.15       
def equation2(input_temp):
    temperature_kelvin = input_temp + Kelvin 
    a_numerator = A_R*np.exp(THETA_A/T_R - THETA_A/temperature_kelvin) 
    a_denominator = 1 + np.exp(THETA_H/T_H - THETA_H/temperature_kelvin)
    return a_numerator/a_denominator

def equation3(input_temp):
    """ Arrhenius equation """
    temperature_kelvin = input_temp + Kelvin
    # Calculate b
    return B_R*np.exp(THETA_B/T_R - THETA_B/temperature_kelvin)

experiment_name = 'SODA'
region = 'NorthAtlantic'

out_dir = f'{Path.cwd()}/growth_model_output/{experiment_name}'
if not os.path.exists(out_dir): os.makedirs(out_dir)

# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False)
# new_time = pd.date_range('2000-01-01', '2005-01-01', freq='M')
# input_files = input_files.assign_coords({'time': new_time})
# input_files = xr.open_mfdataset('gm/input_data/*.nc', decode_times=False) # 2000-2004
input_files = xr.open_dataset('data/SODA_North_Atlantic_1958_2007_sliced.nc')
new_time = pd.date_range('1958-01-01', '2008-01-01', freq='M')
input_files = input_files.assign_coords({'time': new_time})
#input_files['thetao'].transpose('time','latitude','longitude','depth_coord')[0][0][13][0].load()

# Specification of biological parameters  
# Number of years in one life cycle of an individual 
generation = 10
# The year when the input temeperature dataset starts 
# Should be one year less than starting year in the dataset
initial_year = 1960 # 2000    
# Number of years in the input temperature dataset
years = range(initial_year, 2000)  	# range(2000, 2005)  	 

# Define geographic boundaries (latitudes, longitudes)
# Default coordinates: North Atlantic coordinates
#lat_coords = slice(55,56) #slice(51,65)
#lon_coords = slice(0,1) #slice(-3,8)

# Define depth_levels for your growth model output files
depth_levels = slice(30, 600)   # 0-600 meters according to Atlantic cod distribution

lat=55
lon=0
input_files = input_files.sel(latitude=lat, longitude=lon, method='nearest').sel(time=slice(str(years.start),str(years.stop-1)), depth_coord=depth_levels)

# Define latitudes that you will use to save your weight-at-age data to netcdf
#lat = np.array(input_files.latitude, dtype='f')
#lon = np.array(input_files.longitude, dtype='f')

# Define depth that you will use to save your weight-at-age data to netcdf
depths = np.array(input_files.depth_coord, dtype='f')
# Define dimensionality of coords
N_depths, N_lat, N_lon = len(depths), 1,1#len(lat), len(lon)

dt = 1

results = {} # result by year
last_year = initial_year + generation  
while last_year < years[-1]:
    #print(last_year)
    age, weight = 0, np.ones(shape=(N_depths, 1), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, 1), dtype='f')

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(time=str(every_year))            
        
        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths)             
        
        
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:] <= -998)] = np.nan     
        
        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, 1), dtype='f')
        b = np.zeros(shape=(N_depths, 1), dtype='f')
        
        #print(temp_input_3d.shape, temp_input_3d)
        for mon in np.arange(0,12):
            for _ in np.arange(0,30): # day of month
                for ilev in np.arange(0, N_depths):  
                    a[ilev] = equation2(temp_input_3d[mon, ilev])
                    b[ilev] = equation3(temp_input_3d[mon, ilev]) * (-1.)
                    
                    growth_rates[ilev] = 0.01 * ( a[ilev] * weight[ilev]** b[ilev] - C_AVG )  
                    growth_rates[ilev] = np.where(growth_rates[ilev] < 0,0, growth_rates[ilev])
                    
                    weight[ilev] = weight[ilev] * (1. + dt * growth_rates[ilev])
                    
        a_3d = a
        b_3d = b

        growth_rates_3d = growth_rates
        weight_3d = 0.001 * weight 
        weight_max = np.nanmax(weight_3d)

        
        content = {
            'age': age,
            'weight_3d': weight_3d,
            'growth_rates': growth_rates,
            'weight_max': weight_max,
            'a_3d': a_3d,
            'B_3d': b_3d
        }
        
        if f'{every_year}' not in results.keys(): results[f'{every_year}'] = [content]
        else: results[f'{every_year}'].append(content)
        
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 

In [34]:
results['1987']

[{'age': 9,
  'weight_3d': array([[16.442137 ],
         [16.367119 ],
         [15.80209  ],
         [15.224728 ],
         [14.3274145],
         [12.669395 ],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan]], dtype=float32),
  'growth_rates': array([[0.00030169],
         [0.00030613],
         [0.00033871],
         [0.00037327],
         [0.00043008],
         [0.00045922],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan],
         [       nan]], dtype=float32),
  'weight_max': 16.442137,
  'a_3d': array([[5.068994 ],
         [5.0731487],
         [5.077327

In [27]:
import pandas as pd 
# save one location as csv 
# dimensions: time x depth
np.savetxt('data/northsea_lat55_lon0_1960-2000.csv', 
           np.array(pd.DataFrame(np.array(input_files.thetao)).fillna(-999)), 
           delimiter=',')

In [28]:
results = {} # result by year
last_year = initial_year + generation  
while last_year < years[-1]:
    #print(last_year)
    age, weight = 0, np.ones(shape=(N_depths, 1), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, 1), dtype='f')

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(time=str(every_year))            
        
        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths)             
        
        
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:] <= -998)] = np.nan     
        
        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, 1), dtype='f')
        b = np.zeros(shape=(N_depths, 1), dtype='f')
        
        #print(temp_input_3d.shape, temp_input_3d)
        for mon in np.arange(0,12):
            for _ in np.arange(0,30): # day of month
                for ilev in np.arange(0, N_depths):  
                    a[ilev] = equation2(temp_input_3d[mon, ilev])
                    b[ilev] = equation3(temp_input_3d[mon, ilev]) * (-1.)
                    
                    growth_rates[ilev] = 0.01 * ( a[ilev] * weight[ilev]** b[ilev] - C_AVG )  
                    growth_rates[ilev] = np.where(growth_rates[ilev] < 0,0, growth_rates[ilev])
                    
                    weight[ilev] = weight[ilev] * (1. + dt * growth_rates[ilev])


        new_year = int(every_year) + 1
        
        a_3d = a
        b_3d = b

        growth_rates_3d = growth_rates
        weight_3d = 0.001 * weight 
        weight_max = np.nanmax(weight_3d)

        
        content = {
            'age': age,
            'weight_3d': weight_3d,
            'growth_rates': growth_rates,
            'weight_max': weight_max,
            'a_3d': a_3d,
            'B_3d': b_3d
        }
        
        if f'{every_year}' not in results.keys(): results[f'{every_year}'] = [content]
        else: results[f'{every_year}'].append(content)
        
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 

[[0.0514993]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]
 [0.       ]]


ValueError: 

In [24]:
results

{'1960': [{'age': 0,
   'weight_3d': array([[0.36214727],
          [0.34287184],
          [0.31462404],
          [0.27870643],
          [0.23259734],
          [0.1823954 ],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan]], dtype=float32),
   'growth_rates': array([[0.00029162],
          [0.00027096],
          [0.00029427],
          [0.00033974],
          [0.0004277 ],
          [0.00050868],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan],
          [       nan]], dtype=float32),
   'weight_max': 0.36214727,
   'a_3d': array([[5.84

In [ ]:
results = {} # result by year
last_year = initial_year + generation  
while last_year < years[-1]:           
    age, weight = 0, np.ones(shape=(N_depths, N_lat*N_lon), dtype='f')
    growth_rates = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
    age=0

    for every_year in range(initial_year, last_year):
        my_temp = input_files.thetao.sel(time=str(every_year))            
        
        # Partly vectorize 4D temperature fields  to accelerate the computations
        temp_input_3d = my_temp.values.reshape(12, N_depths, N_lat*N_lon)             
            
        # Set NaN values
        temp_input_3d[np.where(temp_input_3d[:,:,:] <= -998)] = np.nan     

        # Initialize necessary variable fields
        a = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')
        b = np.zeros(shape=(N_depths, N_lat*N_lon), dtype='f')

        # mm0 is never used
        if every_year == initial_year: mm0 = 2 
        else: mm0 = 0

        print(temp_input_3d)
        for mon in np.arange(0,12):
            for _ in np.arange(0,30): # day of month
                for ilev in np.arange(0, N_depths):  
                    a[ilev, :] = equation2(temp_input_3d[mon, ilev, :])
                    
                    b[ilev, :] = equation3(temp_input_3d[mon, ilev, :]) * (-1.)
                    growth_rates[ilev, :] = 0.01 * ( a[ilev, :] * weight[ilev, :]** b[ilev, :] - C_AVG )  
                    
                    print(growth_rates.shape, len(growth_rates))
                    raise KeyboardInterrupt
                    
                    growth_rates[ilev, :] = np.where(growth_rates[ilev, :] < 0,0, growth_rates[ilev, :])
                    weight[ilev, :] = weight[ilev, :] * (1. + dt * growth_rates[ilev, :])

        # raise KeyboardInterrupt(44)

        new_year = int(every_year) + 1
        
        # Reshape data to original shape
        a_3d = a.reshape(N_depths, N_lat, N_lon)
        b_3d = b.reshape(N_depths, N_lat, N_lon)

        growth_rates_3d = growth_rates.reshape(N_depths, N_lat, N_lon)

        # # 3D field with asymptotic weight
        weight_3d = 0.001 * weight.reshape((N_depths, N_lat, N_lon)) 
        
        # # Calculate maximum asymptotic weight at a given location 
        # # ("W*" in Butzin and Pörtner (2016)) 
        weight_max = np.nanmax(weight_3d, axis = 0)

        # optional
        content = {
            'age': age,
            'weight_3d': weight_3d,
            'weight_max': weight_max,
            'a_3d': a_3d,
            'B_3d': b_3d
        }
        if f'{every_year}' not in results.keys(): results[f'{every_year}'] = [content]
        else: results[f'{every_year}'].append(content)
        
        # Save netcdf
        # for i, (data, variable_name) in enumerate(zip(
        #     [a_3d, b_3d, growth_rates_3d, weight_3d, weight_max],
        #     ['a_3d', 'b_3d', 'growth_rates_3d', 'weight_3d', 'weight_max'],
        # )):
        #     save2netcdf(
        #         data=data, 
        #         var_name=variable_name, 
        #         directory=f'{out_dir}/init_initial_year_transient' , 
        #         year=every_year, 
        #         age=age, 
        #         dimension_labels=default_dim_labels if i < 4 else  ('latitude', 'longitude')
        #     )
        age += 1
            
    initial_year = initial_year + 1
    last_year = initial_year + generation 
